In [1]:
# 8-1
!mkdir chap8
%cd ./chap8

/Users/maton/BERT_Practice/chap8


/Users/maton/.pyenv/versions/3.10.13/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# 8-3
import itertools
import random
import json
from tqdm import tqdm
import numpy as np
import unicodedata

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForTokenClassification
import pytorch_lightning as pl

# 日本語学習済みモデル
MODEL_NAME = "tohoku-nlp/bert-base-japanese-whole-word-masking"

In [3]:
# 8-4（説明コメント付き：Unicode 正規化 NFKC による表記ゆれの吸収）
# -----------------------------------------------------------------------------
# 目的：
#  - Unicode の **互換性分解 + 合成**（NFKC）で、全角/半角・互換文字（例：半角ｶﾅ・ローマ数字・① 等）
#    を正規化し、検索/重複排除/機械学習前処理での表記ゆれを減らす。
#
# 理論メモ（NFC/NFD/NFKC/NFKD の違い）：
#  - NFC: 正規分解 → 合成（互換性は無視）。見た目を保ちながら正規の合成形式へ。
#  - NFD: 正規分解のみ（結合文字にバラす）。検索・照合の下処理に使うことがある。
#  - NFKC: **互換分解**（見た目は同じでも “意味的に同一視される” 文字を分解）→ 合成。
#          例：全角英数/記号、半角ｶﾅ、ローマ数字 Ⅳ、丸数字 ① などを通常の文字に畳み込む。
#  - NFKD: 互換分解のみ。
# 互換分解は “情報の落ち” が起こり得る（例：①→1、㍍→メートル→m など）。監査用途では注意。
# -----------------------------------------------------------------------------

import unicodedata

# NFKC で正規化する関数
# - 全角英数→半角、半角ｶﾅ→全角カタカナ、結合記号の統合 等を一括で行う
normalize = lambda s: unicodedata.normalize("NFKC", s)

# 動作例：全角/半角の統一（学習前の前処理・検索キー作成などで有効）
print(f'ＡＢＣ -> {normalize("ＡＢＣ")}')  # 全角アルファベット → 半角 "ABC"
print(f'ABC -> {normalize("ABC")}')  # 既に半角 → 変化なし
print(f'１２３ -> {normalize("１２３")}')  # 全角数字 → 半角 "123"
print(f'123 -> {normalize("123")}')  # 既に半角 → 変化なし
print(f'アイウ -> {normalize("アイウ")}')  # 全角カタカナ → 変化なし
print(f'ｱｲｳ -> {normalize("ｱｲｳ")}')  # 半角ｶﾅ → 全角カタカナ "アイウ"

# -----------------------------------------------------------------------------
# 追加の知見（必要ならテストして確認）：
#  - 濁点付き半角ｶﾅ（例：ｶﾞ）も結合が解決され全角「ガ」に統一される：
#      normalize("ｶﾞ") == "ガ"
#  - 互換文字の折り畳み：
#      normalize("ⅠⅡⅢ") -> "III"  （ローマ数字 → ラテン大文字）
#      normalize("①②")   -> "12"   （丸数字 → 通常数字）
#  - 正規化は “可逆でない” ことがあるため、**原文は別項目で保存**しておくと安全。
# -----------------------------------------------------------------------------

ＡＢＣ -> ABC
ABC -> ABC
１２３ -> 123
123 -> 123
アイウ -> アイウ
ｱｲｳ -> アイウ


In [4]:
# 8-5（説明コメント付き：日本語BERT用 NER 前処理ユーティリティ）
# =============================================================================
# 前提：
#  - BertJapaneseTokenizer（transformers）を継承して、固有表現抽出（NER）向けの
#    2種類のエンコード（教師あり/教師なし）と、モデル出力から固有表現スパンを復元する
#    補助関数を提供するクラスである。
#  - ラベリング方式は BIO ではなく「整数 ID によるスパン連結」方式（O=0, その他=タイプID）。
#    → 連続する同一ラベルのトークン列を 1 つのエンティティにまとめる前提。
#  - 重複・ネスト・オーバーラップするエンティティは表現できない（単純スパンのみ）。
#  - prepare_for_model により [CLS]/[SEP] の特殊トークンが付与される前提で、ラベルを先頭/末尾0に整合。
#  - max_length を超える部分は切り捨てられるため、切断されたエンティティは欠落し得る点に注意。
#  - encode_plus_untagged では、MeCab 分かち書き（word_tokenizer）→ WordPiece（subword_tokenizer）
#    で二段階トークナイズを行い、トークン表層を原文へシーケンシャルマッチしてスパンを得る。
#    同一トークンの繰り返しが多い文でも、左から貪欲にマッチしていくため順次対応できる。
#  - return_tensors='pt' 指定時、encoding のみ Tensor 化する（spans はリストのまま返る）。
#    下流でテンソルを期待する場合は適宜変換すること。
# =============================================================================


class NER_tokenizer(BertJapaneseTokenizer):

    def encode_plus_tagged(self, text, entities, max_length):
        """
        文章とそれに含まれる固有表現が与えられた時に、
        符号化とラベル列の作成を行う。

        引数:
          text: str
            元の文章（原文）。エンティティ抽出の対象。
          entities: List[Dict]
            [{'span': [start, end], 'type_id': int}, ...] の形を想定。
            span は原文 text における [start, end) の半開区間、type_id は 0 以外の整数。
          max_length: int
            BERT 入力の最大系列長。prepare_for_model で [CLS]/[SEP] を含む長さに正規化される。

        戻り値:
          encoding: Dict[str, List[int]]
            input_ids / attention_mask / token_type_ids / labels を含む辞書。
            labels は [0]=CLS, 末尾 [0]=SEP/必要に応じ PAD を 0 で詰める。
        """
        # --- 前処理: エンティティを開始位置で昇順ソート ---
        entities = sorted(entities, key=lambda x: x["span"][0])

        # --- 原文を「非固有表現片(O=0)」「固有表現片(=type_id)」に分割して並べる ---
        splitted = []  # 分割後の文字列片を順に蓄積
        position = 0  # 直前に処理した末尾の次インデックス
        for entity in entities:
            start = entity["span"][0]
            end = entity["span"][1]
            label = entity["type_id"]
            # 非固有表現部分（直近の position からエンティティ開始まで）→ ラベル 0
            splitted.append({"text": text[position:start], "label": 0})
            # 固有表現部分（[start:end)）→ ラベル type_id
            splitted.append({"text": text[start:end], "label": label})
            position = end
        # 最後のエンティティ以降の末尾テキストも非固有表現として追加
        splitted.append({"text": text[position:], "label": 0})
        # 空文字は除去（連続するエンティティで start==end 等により空が生じ得るため）
        splitted = [s for s in splitted if s["text"]]

        # --- 片ごとにトークン化し、各トークンへ片に対応するラベルを付与 ---
        tokens = []  # WordPiece トークン列
        labels = []  # トークンに対するラベル（0 or type_id）
        for text_splitted in splitted:
            text = text_splitted["text"]
            label = text_splitted["label"]
            # BertJapaneseTokenizer.tokenize は日本語前処理（MeCab）+ WordPiece を内部で実施
            tokens_splitted = self.tokenize(text)
            labels_splitted = [label] * len(tokens_splitted)
            tokens.extend(tokens_splitted)
            labels.extend(labels_splitted)

        # --- トークン列を ID 列に変換し、BERT 入力辞書へ整形 ---
        input_ids = self.convert_tokens_to_ids(tokens)
        encoding = self.prepare_for_model(
            input_ids, max_length=max_length, padding="max_length", truncation=True
        )  # prepare_for_model が [CLS]/[SEP] 等を付与し、長さを揃える

        # --- ラベルの特殊トークン・PAD 整合 ---
        # 先頭に CLS=0 を付与、本文ラベルは最大長-2（CLS/SEP）までに切詰め、末尾に SEP=0
        labels = [0] + labels[: max_length - 2] + [0]
        # さらに不足分（PAD 部分）を 0 で埋め、最終的に長さを max_length に一致させる
        labels = labels + [0] * (max_length - len(labels))
        encoding["labels"] = labels

        return encoding

    def encode_plus_untagged(self, text, max_length=None, return_tensors=None):
        """
        文章をトークン化し、それぞれのトークンの文章中の位置も特定しておく。

        引数:
          text: str
            元の文章。
          max_length: Optional[int]
            指定時は padding='max_length', truncation=True で固定長化。
            未指定時は可変長（特殊トークン付与は維持）。
          return_tensors: Optional[str]
            'pt' を指定すると encoding の各値を torch.Tensor(batch次元つき) に変換。

        戻り値:
          encoding: Dict[str, List[int]] or Dict[str, torch.Tensor]
            prepare_for_model の出力（[CLS]/[SEP] 付与後）。
          spans: List[List[int]]
            各トークンに対応する原文上の [start, end) 位置。
            特殊トークン([CLS],[SEP],[PAD])の位置は [-1,-1] のダミーにする。
        """
        # --- 形態素（MeCab）→ サブワード（WordPiece）の二段トークナイズ ---
        tokens = []  # WordPiece トークン
        tokens_original = (
            []
        )  # スパン計算用の「表層文字列」列（'##' 除去/UNK は単語そのまま）
        words = self.word_tokenizer.tokenize(text)  # MeCab による単語列
        for word in words:
            # 単語を WordPiece に分割
            tokens_word = self.subword_tokenizer.tokenize(word)
            tokens.extend(tokens_word)
            if (
                tokens_word[0] == "[UNK]"
            ):  # 未知語は WordPiece に分割できない → 原単語をそのまま使う
                tokens_original.append(word)
            else:
                # '##' 接頭辞を削って表層形を復元（スパン同定に用いる）
                tokens_original.extend(
                    [token.replace("##", "") for token in tokens_word]
                )

        # --- 原文上のスパンを左から順次マッチで同定（空白/記号も考慮して進める）---
        position = 0  # 原文上の走査位置（左から前進のみ）
        spans = []  # 各トークンの [start, end) を蓄積
        for token in tokens_original:
            l = len(token)
            while 1:
                # 現在位置から長さ l を切り出して一致判定（空白等を飛ばすため不一致なら1文字進める）
                if token != text[position : position + l]:
                    position += 1
                else:
                    spans.append([position, position + l])
                    position += l
                    break
        # ここでの手法は「逐次マッチ」のため、同一部分文字列の繰り返しがあっても左から順に整合が取れる。
        # ただし、原文編集（正規化や空白折り畳み）を別途行う場合は、同じ変換を token 側にも適用しておくこと。

        # --- BERT 入力辞書へ整形（固定長/可変長を分岐）---
        input_ids = self.convert_tokens_to_ids(tokens)
        encoding = self.prepare_for_model(
            input_ids,
            max_length=max_length,
            padding="max_length" if max_length else False,
            truncation=True if max_length else False,
        )
        sequence_length = len(
            encoding["input_ids"]
        )  # [CLS]/[SEP] を含む最終トークン列長

        # --- 特殊トークン分のダミー span を付与し、長さを encoding に一致させる ---
        # 先頭 [CLS] 用のダミー
        spans = [[-1, -1]] + spans[: sequence_length - 2]
        # 末尾 [SEP] と PAD 分をダミーで埋める（長さを sequence_length に揃える）
        spans = spans + [[-1, -1]] * (sequence_length - len(spans))

        # --- 必要なら Tensor 化（encoding のみ。spans はリストで返す） ---
        if return_tensors == "pt":
            encoding = {k: torch.tensor([v]) for k, v in encoding.items()}

        return encoding, spans

    def convert_bert_output_to_entities(self, text, labels, spans):
        """
        文章、ラベル列の予測値、各トークンの位置から固有表現を得る。

        引数:
          text: str
            原文。
          labels: Sequence[int]
            各トークンのラベル（O=0, その他は type_id）。[CLS]/[SEP]/[PAD] 位置の分も含む想定。
          spans: Sequence[List[int]]
            各トークンの [start, end)。特殊トークンは [-1,-1]。

        戻り値:
          entities: List[Dict]
            {"name": 原文片, "span": [start,end], "type_id": ラベルID} の配列。
        """
        # --- 特殊トークン（span=-1）に対応するラベル/スパンを除去して本文トークンに限定 ---
        labels = [label for label, span in zip(labels, spans) if span[0] != -1]
        spans = [span for span in spans if span[0] != -1]

        # --- 連続する同一ラベルをまとめて 1 スパンのエンティティに復元 ---
        # itertools.groupby により、labels の連続区間ごとにグルーピングする。
        # 例: labels=[0,0,2,2,0,3] → (0区間)(2区間)(0区間)(3区間)
        # ラベル0（O）は無視し、非0の区間のみエンティティを生成する。
        entities = []
        for label, group in itertools.groupby(enumerate(labels), key=lambda x: x[1]):

            group = list(group)  # group は [(idx,label), ...] の列
            start = spans[group[0][0]][0]  # 区間先頭トークンの start
            end = spans[group[-1][0]][1]  # 区間末尾トークンの end（半開区間の終端）

            if label != 0:  # O 以外のラベルのみエンティティ化
                entity = {
                    "name": text[start:end],  # 原文断片（復元テキスト）
                    "span": [start, end],  # 原文上の [start,end)
                    "type_id": label,  # ラベル ID（BIO ではなく type_id そのもの）
                }
                entities.append(entity)

        # 注意：
        #  - BIO 方式ではないため、同一ラベルが連続すれば 1 エンティティに連結される。
        #    同じ type_id が離れて複数回出現する場合は、それぞれ別エンティティとして復元される。
        #  - ネスト/オーバーラップは扱えない（単純連続区間のみ）。
        #  - max_length による切詰めでエンティティの一部が失われると、復元されない場合がある。

        return entities

In [5]:
# 8-6
tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.


In [6]:
# 8-7（説明コメント付き：NER_tokenizer.encode_plus_tagged の動作確認）
# =============================================================================
# 目的：
#  - 原文 `text` と、原文上のスパン（[start, end) の半開区間）＋ラベルIDからなる `entities`
#    を与え、BERT 入力（input_ids/attention_mask/token_type_ids）に整形すると同時に、
#    各トークンに対応する **ラベル列**（O=0, エンティティは type_id）を作成する。
#
# 理論メモ：
#  - encode_plus_tagged は「BIO 方式」ではなく **連結方式**（同一ラベルが連続するトークン列を1つの
#    エンティティとみなす）前提でラベルを作る。特殊トークン [CLS]/[SEP]/[PAD] は 0 として埋める。
#  - `span=[start,end]` は **半開区間**（start 文字を含み end 文字を含まない）。インデックスは 0 始まり。
#  - トークン化は WordPiece なので、1語が複数トークンに分割されても、分割前の片に付けたラベルが
#    そのまま各トークンに複製される設計。
#  - `max_length` 超過分は切り捨てられるため、末尾側のエンティティが途中で切れると、対応トークンに
#    ラベルが乗らず欠落し得る点に注意（長さ設計 or スライディングで対処）。
#  - 直前に正規化（例：NFKC）を行った場合、**スパンは正規化後の文字列基準**で与えること（前後で
#    文字長が変わるとズレる）。
# =============================================================================

# （互換/保守）NER_tokenizer が未インポート/未定義でも動くように最小限のフォールバックを用意
try:
    NER_tokenizer
except NameError:
    # ここではクラス本体は 8-5 で定義済み想定。未定義なら明示エラーにする。
    raise RuntimeError(
        "NER_tokenizer クラスが未定義です。先に 8-5 の定義を実行してください。"
    )

# 既存の `tokenizer` が NER_tokenizer でない場合は、同名で置き換えても構わない運用なら差し替える
try:
    tokenizer
    has_encode_tagged = hasattr(tokenizer, "encode_plus_tagged")
except NameError:
    has_encode_tagged = False

if not has_encode_tagged:
    # 学習時と同じ語彙を使う（未定義なら東北大BERTにフォールバック）
    try:
        MODEL_NAME
    except NameError:
        MODEL_NAME = "tohoku-nlp/bert-base-japanese-whole-word-masking"
    tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

# --------------------------- 元のコード（＋説明コメント） ---------------------------

# 入力テキスト（インデックスは 0 始まり）
# 文字位置: 0:昨,1:日,2:の,3:み,4:ら,5:い,6:事,7:務,8:所,9:と,10:の,11:打,12:ち,13:合,14:わ,15:せ,16:は,17:順,18:調,19:だ,20:っ,21:た,22:。
text = "昨日のみらい事務所との打ち合わせは順調だった。"

# エンティティの指定：
#  - name は人間可読の補助で、実際のラベル付与は span と type_id によって行われる
#  - span=[3,9) → 原文中の「みらい事務所」（3〜8文字目）を指定（end=9 は「と」なので非包含）
#  - type_id=1 → O（背景）を 0 とするクラス設計の「1番」カテゴリ
entities = [{"name": "みらい事務所", "span": [3, 9], "type_id": 1}]

# ラベル列作成つきの符号化（最大長は [CLS]/[SEP] を含むトークン長に正規化される）
# labels は先頭[CLS]=0、本文（max_length-2 まで）、末尾[SEP]=0、残りPAD=0 で埋められる。
encoding = tokenizer.encode_plus_tagged(text, entities, max_length=20)

# 出力の中身は Hugging Face の標準キー（input_ids, attention_mask, token_type_ids）に加え、
# 1:1 に対応する labels が入っている。確認用にそのまま表示。
print(encoding)

# 追加の確認（任意）：
# - トークン列長とラベル列長が一致していること
# - ラベル中の 1 の連続区間が「みらい事務所」の WordPiece 分割数に一致していること
# 例：
# tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])
# print(tokens)
# print(encoding["labels"])

{'input_ids': [2, 10271, 28486, 5, 546, 3000, 1518, 233, 13, 5, 1878, 2682, 9, 10750, 308, 10, 8, 3, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], 'labels': [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [7]:
# 8-8
text = "騰訊の英語名はTencent Holdings Ltdである。"
encoding, spans = tokenizer.encode_plus_untagged(text, return_tensors="pt")
print("# encoding")
print(encoding)
print("# spans")
print(spans)

# encoding
{'input_ids': tensor([[    2,     1,     5,  1543,   125,     9,  6749, 28550,  2953, 28550,
         28566, 21202, 28683, 14050, 12475,    12,    31,     8,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
# spans
[[-1, -1], [0, 2], [2, 3], [3, 5], [5, 6], [6, 7], [7, 9], [9, 10], [10, 12], [12, 13], [13, 14], [15, 18], [18, 19], [19, 23], [24, 27], [27, 28], [28, 30], [30, 31], [-1, -1]]


In [8]:
# 8-9
labels_predicted = [0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
entities = tokenizer.convert_bert_output_to_entities(text, labels_predicted, spans)
print(entities)

[{'name': '騰訊の', 'span': [0, 3], 'type_id': 1}, {'name': 'ncent Holdings Ltdで', 'span': [9, 28], 'type_id': 1}]


In [9]:
# 8-10（説明コメント付き：日本語BERTをトークン分類（NER）用に初期化）
# =============================================================================
# 目的：
#  - 日本語BERTの語彙・前処理（MeCab + WordPiece）に対応した **NER_tokenizer** を読み込み、
#    事前学習モデルの上に **BertForTokenClassification** ヘッド（num_labels=4）を載せる。
#  - MacBook 環境（Apple Silicon/MPS）・CUDA・CPU のいずれでも動くように **デバイス自動選択**で配置。
#
# 理論メモ：
#  - Token Classification は各トークン t に対してクラスのロジット z_{t,c} を出力し、
#    学習時は通常 CrossEntropyLoss（各位置の多クラス）を用いる。
#  - NER ではしばしば **BIO/IOBES** を採用するが、本書式は「O=0, エンティティ種別ID>0」
#    のシンプル方式（連結方式）も可能。`num_labels=4` は例として
#       0: "O", 1: "ORG", 2: "PER", 3: "LOC"
#    のように解釈できる（実際の定義はデータに合わせて固定・共有すること）。
#  - 学習時に `[CLS]/[SEP]/[PAD]` 位置のラベルを損失から除外するには、
#    データ側のラベルを **ignore_index（例：-100）** にしておくのが Hugging Face の定石。
# =============================================================================

import torch
from transformers import BertForTokenClassification

# --- （参考）元のコード（実行はしない。可搬性の観点で .cuda() 固定は避けるため） ---
# tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)
# bert_tc = BertForTokenClassification.from_pretrained(
#     MODEL_NAME, num_labels=4
# )
# bert_tc = bert_tc.cuda()

# --- MODEL_NAME が未定義なら東北大BERTにフォールバック ---
try:
    MODEL_NAME
except NameError:
    MODEL_NAME = "tohoku-nlp/bert-base-japanese-whole-word-masking"

# --- NER_tokenizer クラスの存在確認（8-5 で定義済み想定） ---
try:
    NER_tokenizer
except NameError as e:
    raise RuntimeError(
        "NER_tokenizer クラスが未定義です。先に 8-5 の定義を実行してください。"
    ) from e

# --- ラベル定義（例）。実データに合わせて **固定・共有** することが重要 ---
num_labels = 4
id2label = {0: "O", 1: "ORG", 2: "PER", 3: "LOC"}  # 例：組織/人/場所
label2id = {v: k for k, v in id2label.items()}


# --- デバイス自動選択（Mac MPS → CUDA → CPU） ---
def pick_device() -> torch.device:
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return torch.device("mps")
    if torch.cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")


device = pick_device()
print(f"[info] device = {device}")

# --- トークナイザ：日本語用の NER_tokenizer（MeCab + WordPiece 連携） ---
tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

# --- モデル本体：事前学習BERT + Token Classification ヘッド ---
bert_tc = (
    BertForTokenClassification.from_pretrained(
        MODEL_NAME,
        num_labels=num_labels,
        id2label=id2label,  # 推論・可視化で役立つ（config に保存され ckpt へも残る）
        label2id=label2id,
    )
    .to(device)
    .eval()
)  # 推論時は eval()。学習時は train() に切替。

# =============================================================================
# 補足（実務向け）：
#  - 学習：
#      - ラベル列の特殊トークン位置は `-100`（ignore_index）にして DataCollator などで結合。
#      - Optimizer は AdamW、Scheduler は linear/warmup を推奨。
#  - 推論：
#      - `outputs = bert_tc(**batch)` → `logits.argmax(-1)` でラベルID列。
#      - `tokenizer` から `offset_mapping`（Fast版）を使える場合、原文スパン復元が堅牢。
#  - 再現性：
#      - `MODEL_NAME`、`id2label`、`max_length`、正規化の有無（NFKC）を **明示的に固定**・記録。
# =============================================================================

[info] device = mps


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at tohoku-nlp/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# 8-12（修正版：CUDA 未有効エラーの解消と実務的ガードを追加）
# =============================================================================
# エラー原因：
#   AssertionError: Torch not compiled with CUDA enabled
# → 環境が CUDA 非対応なのに `.cuda()` を呼んだため。
#
# 対処方針：
#  1) モデルが載っている実デバイスに合わせて **.to(device)** に統一（MPS/CUDA/CPU どれでも可）。
#  2) （任意だが推奨）特殊トークン([CLS]/[SEP]/[PAD])と PAD 部分のラベルを **-100** にして
#     CrossEntropyLoss の ignore_index に合わせ、損失に寄与させない。
#     ※ 8-5 実装はこれらを 0=O にしているため、何もしないと O クラス過多で学習が歪む。
# =============================================================================

import torch
from torch.utils.data import DataLoader

# --- 手作り NER データ（元コードそのまま） ---
data = [
    {
        "text": "AさんはB大学に入学した。",
        "entities": [
            {"name": "A", "span": [0, 1], "type_id": 2},
            {"name": "B大学", "span": [4, 7], "type_id": 1},
        ],
    },
    {
        "text": "CDE株式会社は新製品「E」を販売する。",
        "entities": [
            {"name": "CDE株式会社", "span": [0, 7], "type_id": 1},
            {"name": "E", "span": [12, 13], "type_id": 3},
        ],
    },
]

# --- 符号化：8-5 の encode_plus_tagged を使用（元コードそのまま） ---
max_length = 32
dataset_for_loader = []
for sample in data:
    text = sample["text"]
    entities = sample["entities"]
    encoding = tokenizer.encode_plus_tagged(text, entities, max_length=max_length)
    # HF の TokenClassification は labels を long テンソル（クラスID）で受けるのが定石
    encoding = {k: torch.tensor(v) for k, v in encoding.items()}
    dataset_for_loader.append(encoding)

# --- DataLoader（元コードの方針：全件を 1 ミニバッチに） ---
dataloader = DataLoader(dataset_for_loader, batch_size=len(data))

# --- デバイス：モデル側に合わせて自動決定（.cuda() を撤廃） ---
device = next(bert_tc.parameters()).device  # 例：mps/cuda/cpu のいずれか


# --- 推奨：特殊トークンと PAD を損失から除外（ignore_index=-100）に置換する関数 ---
def mask_special_positions_to_ignore_index(batch_dict, tokenizer, ignore_index=-100):
    """
    TokenClassification の損失から除外したい位置（[CLS]/[SEP]/[PAD] と attention_mask=0）を
    ラベル上で ignore_index に置き換える。
    """
    input_ids = batch_dict["input_ids"]
    attention_mask = batch_dict["attention_mask"]
    labels = batch_dict["labels"]

    # PAD 位置（attention_mask==0）を -100 に
    labels = labels.masked_fill(attention_mask.eq(0), ignore_index)

    # CLS/SEP 位置を -100 に（存在チェック込み）
    cls_id = tokenizer.cls_token_id
    sep_id = tokenizer.sep_token_id
    for i in range(labels.size(0)):
        cls_pos = (input_ids[i] == cls_id).nonzero(as_tuple=True)[0]
        sep_pos = (input_ids[i] == sep_id).nonzero(as_tuple=True)[0]
        if cls_pos.numel() > 0:
            labels[i, cls_pos] = ignore_index
        if sep_pos.numel() > 0:
            labels[i, sep_pos] = ignore_index

    batch_dict["labels"] = labels
    return batch_dict


# --- ミニバッチで損失を計算（.to(device) で可搬化） ---
for batch in dataloader:
    # 元コードの `.cuda()` を **.to(device)** に差し替え（MPS/CPU でも動く）
    batch = {k: v.to(device) for k, v in batch.items()}

    # （任意・推奨）損失に含めない位置を ignore_index=-100 に置換
    batch = mask_special_positions_to_ignore_index(batch, tokenizer, ignore_index=-100)

    # 推論または学習（ここでは損失取得のみ）
    output = bert_tc(**batch)  # labels が含まれているため内部で CrossEntropyLoss を計算
    loss = output.loss  # 平均化されたトークン単位 CE 損失
    print(f"loss = {loss.item():.4f}")

# =============================================================================
# 理論補足：
#  - CrossEntropyLoss はクラス ID（long）を取り、ignore_index（既定 -100）位置は損失計算から除外。
#  - ignore_index を適用すると、PAD や特殊トークンの頻度に損失が引っぱられず、学習が安定。
#  - 連結方式（O=0, type_id>0）でも動作するが、隣接同タイプを分離したいなら BIO/IOBES へ拡張。
#  - 実学習では bert_tc.train()、評価や予測では bert_tc.eval() + no_grad() を併用する。
# =============================================================================

loss = 1.4404


In [12]:
!git clone --branch v2.0 https://github.com/stockmarkteam/ner-wikipedia-dataset 

Cloning into 'ner-wikipedia-dataset'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 35 (delta 11), reused 11 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (35/35), 664.15 KiB | 655.00 KiB/s, done.
Resolving deltas: 100% (11/11), done.
Note: switching to 'f7ed83626d90e5a79f1af99775e4b8c6cba15295'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



In [13]:
# 8-14
# データのロード
dataset = json.load(open("ner-wikipedia-dataset/ner.json", "r"))
# ↑ 事前作成済みの NER 用 JSON を読み込む。
#    想定フォーマット：
#      sample = {
#        "text": <文字列>,
#        "entities": [
#           {"name": <表層>, "span": [start, end], "type": <カテゴリ名>}, ...
#        ]
#      }
#    ※ 文字インデックス span は 0 始まりの半開区間 [start, end) を想定。
#    ※ エンコード（UTF-8 など）に依存する場合は open(..., encoding='utf-8') を明示すると安全。

# 固有表現のタイプとIDを対応付る辞書
type_id_dict = {
    "人名": 1,
    "法人名": 2,
    "政治的組織名": 3,
    "その他の組織名": 4,
    "地名": 5,
    "施設名": 6,
    "製品名": 7,
    "イベント名": 8,
}
# ↑ 学習・推論で一貫して使う「カテゴリ名 → 整数ID」対応。
#    ・TokenClassification/BIO 系では num_labels ≥ 最大ID+1 が必要。
#    ・この対応はメタデータとして保存（JSON 等）し、学習・推論で共有すること。

# カテゴリーをラベルに変更、文字列の正規化する。
for sample in dataset:
    sample["text"] = unicodedata.normalize("NFKC", sample["text"])
    # ↑ NFKC（互換分解＋合成）で全角/半角・互換文字の表記ゆれを統一。
    #    【重要】NFKC は “非可逆” かつ “文字長が変化” することがあるため、
    #    もし `entities[*]['span']` が「正規化前の text に基づく」場合は
    #    span がズレる＝無効になる点に注意。
    #    → 本コードは span を再計算していないため、
    #       * データが “すでに正規化済み” である
    #       * または 正規化後の text を基準に span が定義されている
    #      ことが前提。そうでない場合は offset を再構築する前処理が必要（TODO）。

    for e in sample["entities"]:
        e["type_id"] = type_id_dict[e["type"]]
        del e["type"]
        # ↑ 学習で扱うのは数値ラベル（type_id）。人可読名（type）は破棄。
        #    推論結果の可視化用に name/type 名称が必要なら、別の辞書を持つか
        #    id2type で復号できるようにしておくこと。

# データセットの分割
random.shuffle(dataset)
# ↑ シャッフルしてから split。再現性を担保したい場合は予め random.seed(固定値) を設定する。
n = len(dataset)
n_train = int(n * 0.6)
n_val = int(n * 0.2)
dataset_train = dataset[:n_train]
dataset_val = dataset[n_train : n_train + n_val]
dataset_test = dataset[n_train + n_val :]
# ↑ 単純比率 split（60/20/20）。
#    ・ラベル分布が偏る可能性があるため、実務では層化分割（stratified split）を推奨。
#    ・文長分布やエンティティ有無の偏りも評価指標に影響するため、分割前に統計確認を行うと良い。

In [14]:
# 8-15（説明コメント付き：NER データのエンコード→DataLoader 準備）
# =============================================================================
# 目的：
#  - 文字スパン付き NER データ（sample = {'text': str, 'entities': [{'span':[s,e], 'type_id':k}, ...] }）を
#    学習時に直接使える形（BERT 入力辞書＋トークンラベル）へ変換し、DataLoader を構築する。
#
# 理論メモ：
#  - encode_plus_tagged（8-5で定義）は、原文をエンティティ境界で分割 → WordPiece 化 →
#    片ラベル（O=0 / type_id>0）を各トークンへ複写 → [CLS]/[SEP]/PAD を付与して長さ max_length に整形する。
#  - Token Classification の学習では **CrossEntropyLoss** を用いるため、labels は **クラスID（LongTensor）** を渡すのが定石。
#  - [CLS]/[SEP]/PAD 等の位置は損失から除外するのが一般的（ignore_index=-100）。この関数では 0=O のまま作るので、
#    学習ループ側でマスクに置き換える（または DataCollator で処理）と良い。
#  - max_length を越える部分は切り捨てられるため、右端でエンティティが途切れる場合がある。実務では
#    スライディングウィンドウや長めの max_length を検討する。
# =============================================================================


def create_dataset(tokenizer, dataset, max_length):
    """
    データセットをデータローダに入力できる形に整形。

    引数:
      tokenizer : NER_tokenizer
        8-5 で定義した encode_plus_tagged を持つトークナイザ（語彙は学習・推論で固定）。
      dataset   : List[Dict]
        [{'text': str, 'entities': [{'span':[s,e], 'type_id':k}, ...]}, ...]
      max_length: int
        BERT 入力系列の上限（[CLS]/[SEP] を含む長さに正規化）。

    戻り値:
      dataset_for_loader : List[Dict[str, torch.Tensor]]
        input_ids/attention_mask/token_type_ids/labels を LongTensor で持つ辞書のリスト。
        DataLoader で自動的に key ごとにスタックされミニバッチになる。
    """
    dataset_for_loader = []
    for sample in dataset:
        text = sample["text"]
        entities = sample["entities"]

        # 文字スパン→トークンラベル写像（O=0 / type_id>0）。[CLS]/[SEP]/PAD も付与され長さを揃える。
        encoding = tokenizer.encode_plus_tagged(text, entities, max_length=max_length)

        # HF の TokenClassification は labels を「整数クラスID（Long）」で受けるのが定石。
        # attention_mask/input_ids/token_type_ids も Long で問題ない（CE計算は labels の型が重要）。
        # ※ 特殊トークン/PAD を損失から除外したい場合は、学習ループ側で -100 に置換すること。
        encoding = {k: torch.tensor(v) for k, v in encoding.items()}

        dataset_for_loader.append(encoding)

    return dataset_for_loader


# トークナイザのロード
# - 事前学習モデルの語彙・前処理（MeCab + WordPiece）と一致させること。
# - MODEL_NAME は学習・推論を通じて固定し、再現性のためにメタとして保存しておく。
tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

# データセットの作成
# - 学習安定の観点で 128 などの固定長を採用（動的パディングよりスループットが安定しやすい）。
# - 文が長く切れやすいコーパスでは 256/384/512 も検討（VRAM とトレードオフ）。
max_length = 128
dataset_train_for_loader = create_dataset(tokenizer, dataset_train, max_length)
dataset_val_for_loader = create_dataset(tokenizer, dataset_val, max_length)

# データローダの作成
# - 学習は shuffle=True（i.i.d. 近似と汎化のため）。検証/テストは順序維持で十分。
# - 実務では num_workers / pin_memory（CUDA時）や DataCollatorForTokenClassification の導入を検討。
# - デバイスへの転送は学習ループ側で `batch = {k: v.to(device) for k,v in batch.items()}` とする。
dataloader_train = DataLoader(dataset_train_for_loader, batch_size=32, shuffle=True)
dataloader_val = DataLoader(dataset_val_for_loader, batch_size=256)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.


In [ ]:
# 8-16（説明コメント付き：Lightning v2 対応・可搬デバイス・損失マスク（CLS/SEP/PAD 除外））

import torch
import pytorch_lightning as pl
from transformers import BertForTokenClassification, AutoTokenizer

# =============================================================================
# 目的
#  - TokenClassification 用の LightningModule を作成し、学習/検証の損失をロギングする。
#  - Lightning v2 で非推奨の `gpus=1` を廃止し、`accelerator`/`devices` を用いた可搬設定に修正。
#  - 学習の健全性向上のため、損失計算から [CLS]/[SEP]/PAD を除外（ignore_index 相当のマスク）する。
#
# 背景理論（要点）
#  - トークン分類は位置 t ごとに C クラスのロジット z_{t,c} を出力し、CrossEntropyLoss を計算する。
#  - 一般に PAD や特殊トークン（CLS/SEP）は学習に寄与させない（HF の定石は labels を -100 にする）。
#  - 本実装では、バッチ受領後に labels を複製し、attention_mask==0 と CLS/SEP 位置を -100 に置換してから
#    BertForTokenClassification に渡す。（= ignore_index 処理）
# =============================================================================


class BertForTokenClassification_pl(pl.LightningModule):
    def __init__(self, model_name, num_labels, lr):
        """
        model_name: 事前学習BERT（例：tohoku-nlp/bert-base-japanese-whole-word-masking）
        num_labels: クラス数（O=0 を含む総数）
        lr       : 学習率（微調整の目安 1e-5〜5e-5）
        """
        super().__init__()
        self.save_hyperparameters()

        # 本体モデル（トークン分類ヘッド付き）
        self.bert_tc = BertForTokenClassification.from_pretrained(
            model_name,
            num_labels=num_labels,
        )

        # CLS/SEP のトークンIDを取得（損失から除外するために使用）
        tok = AutoTokenizer.from_pretrained(model_name)
        self.cls_id = tok.cls_token_id
        self.sep_id = tok.sep_token_id

    # -------------------- 内部ユーティリティ：損失無視マスクを埋める --------------------
    def _apply_ignore_index_mask(self, batch, ignore_index=-100):
        """
        CrossEntropyLoss の対象外にしたい位置（PAD/CLS/SEP）を labels 上で ignore_index に置換する。
        - PAD は attention_mask==0 で同定。
        - CLS/SEP は input_ids の値から同定。
        """
        if "labels" not in batch:
            return batch  # 推論時など labels が無い場合はそのまま返す

        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"].clone()  # 破壊的変更を避けるため複製

        # PAD 位置（attention_mask==0）を ignore_index に
        labels = labels.masked_fill(attention_mask.eq(0), ignore_index)

        # CLS/SEP の位置も ignore_index に
        # 例外的に CLS/SEP が存在しないトークナイザ構成は想定外。存在チェックしつつ置換。
        if self.cls_id is not None:
            cls_pos = input_ids.eq(self.cls_id)
            labels = labels.masked_fill(cls_pos, ignore_index)
        if self.sep_id is not None:
            sep_pos = input_ids.eq(self.sep_id)
            labels = labels.masked_fill(sep_pos, ignore_index)

        batch = dict(batch)
        batch["labels"] = labels
        return batch

    # -------------------- 学習/検証フロー --------------------
    def training_step(self, batch, batch_idx):
        # 損失から除外すべき位置を -100 に置換してから forward
        batch = self._apply_ignore_index_mask(batch, ignore_index=-100)
        output = self.bert_tc(**batch)
        loss = output.loss
        # エポック平均で可視化（prog_bar=True で進捗バーにも表示）
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        batch = self._apply_ignore_index_mask(batch, ignore_index=-100)
        output = self.bert_tc(**batch)
        val_loss = output.loss
        self.log("val_loss", val_loss, on_step=False, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        # 最小構成：Adam（実務は AdamW + Scheduler を推奨）
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)


# -------------------- チェックポイント（val_loss 最小） --------------------
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor="val_loss",
    mode="min",
    save_top_k=1,
    save_weights_only=True,  # 版差あり：必要に応じて state_dict 手動保存を検討
    dirpath="model/",
    filename="epoch={epoch}-val_loss={val_loss:.4f}",
)

# -------------------- Trainer（Lightning v2 形式に修正：gpus→accelerator/devices） --------------------
# 可搬設定：CUDA があれば GPU、Apple Silicon なら MPS、無ければ CPU を自動選択
trainer = pl.Trainer(
    accelerator="auto",
    devices=1,
    max_epochs=5,
    callbacks=[checkpoint],
    # deterministic=True,              # 再現性重視（性能とトレードオフ）
    # gradient_clip_val=1.0,           # 勾配クリップ（安定化）
    # accumulate_grad_batches=2,       # 勾配蓄積（VRAM 節約）
    # precision=16,                    # 混合精度（環境に応じて）
)

# -------------------- ファインチューニング実行 --------------------
model = BertForTokenClassification_pl(MODEL_NAME, num_labels=9, lr=1e-5)

# DataLoader は 8-15 で作成済み（dataloader_train / dataloader_val）
trainer.fit(model, dataloader_train, dataloader_val)

# 最良モデル（val_loss 最小）のファイルパス
best_model_path = checkpoint.best_model_path
print(f"[best ckpt] {best_model_path}")

# 参考：ロード例
# loaded = BertForTokenClassification_pl.load_from_checkpoint(best_model_path)
# loaded.eval()